In [13]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [9]:
data=pd.read_csv('WhoScoredFinal1.csv', delimiter=',')

dataForward=data[(data['positionText']=='Forwad')|(data['positionText']=='Midfielder')][['playerId','lastName','assistTotal', 'keyPassesTotal', 'dribbleWon', 'dispossessedPerGame','goalTotal','rating']]
dataForwardFeatures=dataForward.groupby('playerId')['assistTotal','keyPassesTotal','dribbleWon', 'dispossessedPerGame','goalTotal'].mean().fillna(0)
dataForwardLabels=dataForward.groupby('playerId')['rating'].mean().fillna(0)

<ipython-input-9-6ddf2da35ec4>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataForwardFeatures=dataForward.groupby('playerId')['assistTotal','keyPassesTotal','dribbleWon', 'dispossessedPerGame','goalTotal'].mean().fillna(0)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

trainFeatures, testFeatures, trainLables, testLabels = train_test_split(dataForwardFeatures, dataForwardLabels, test_size=0.25)

min_max_scaler = preprocessing.MinMaxScaler()
trainFeaturesScaled = pd.DataFrame(min_max_scaler.fit_transform(trainFeatures))
testFeaturesScaled = pd.DataFrame(min_max_scaler.fit_transform(testFeatures))

trainLablesScaled = pd.DataFrame(min_max_scaler.fit_transform(trainLables.to_numpy().reshape(-1, 1)))
testLabelsScaled = pd.DataFrame(min_max_scaler.fit_transform(testLabels.to_numpy().reshape(-1, 1)))

In [11]:
dataForwardLabels

playerId
3         6.082955
11        7.164530
17        7.041550
29        6.105714
48        7.195092
            ...   
393392    6.310000
393393    6.390000
393395    6.230000
393562    5.960000
394786    6.430000
Name: rating, Length: 9161, dtype: float64

In [30]:
# MLP

modelMLP = Sequential()
modelMLP.add(Dense(5, input_shape=(5, ), activation='relu'))
modelMLP.add(Dense(3, activation='relu'))
modelMLP.add(Dense(1, activation='relu'))
modelMLP.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 4         
Total params: 52
Trainable params: 52
Non-trainable params: 0
_________________________________________________________________


In [35]:
modelMLP.compile(loss='mean_squared_error',
              optimizer= 'adam',
              metrics=['mean_squared_error'])

historyMLP = modelMLP.fit(trainFeaturesScaled, trainLables, epochs=10, batch_size=1000) #, validation_data=(testFeaturesScaled, testLabels))

Epoch 1/10
7/7 [==============================] - 0s 1ms/step - loss: 40.6197 - mean_squared_error: 40.6197
Epoch 2/10
7/7 [==============================] - 0s 1ms/step - loss: 40.3771 - mean_squared_error: 40.3771
Epoch 3/10
7/7 [==============================] - 0s 1ms/step - loss: 40.2149 - mean_squared_error: 40.2149
Epoch 4/10
7/7 [==============================] - 0s 1ms/step - loss: 40.0198 - mean_squared_error: 40.0198
Epoch 5/10
7/7 [==============================] - 0s 999us/step - loss: 39.8041 - mean_squared_error: 39.8041
Epoch 6/10
7/7 [==============================] - 0s 1ms/step - loss: 39.5271 - mean_squared_error: 39.5271
Epoch 7/10
7/7 [==============================] - 0s 1ms/step - loss: 39.3218 - mean_squared_error: 39.3218
Epoch 8/10
7/7 [==============================] - 0s 1ms/step - loss: 39.0490 - mean_squared_error: 39.0490
Epoch 9/10
7/7 [==============================] - 0s 1ms/step - loss: 38.8025 - mean_squared_error: 38.8025
Epoch 10/10
7/7 [=========

In [33]:
predicted = modelMLP.predict(testFeaturesScaled)
# predicted
indexes = np.array(testLabels.index)
indexes = indexes.reshape(2291,1)
labels = np.array(testLabels).reshape(2291,1)
np.concatenate((indexes,labels,predicted), axis=1)

array([[7.40690000e+04, 6.37963158e+00, 1.24201551e-01],
       [2.06590000e+04, 6.03500000e+00, 1.42607167e-01],
       [2.30500000e+03, 6.84576923e+00, 2.16048896e-01],
       ...,
       [3.55779000e+05, 6.73347692e+00, 1.43596664e-01],
       [2.71619000e+05, 6.08633333e+00, 1.33410990e-01],
       [3.28610000e+04, 6.79731782e+00, 1.66336715e-01]])

In [34]:
modelMLP.evaluate(testFeaturesScaled, testLabelsScaled)

72/72 [==============================] - 0s 535us/step - loss: 0.1275 - mean_squared_error: 0.1275


[0.1275159865617752, 0.1275159865617752]